<!-- ![title](Figures/cat2.png) -->
# PURR Processing Utility for Rapid Results

Import all necessary libraries

In [1]:
#Import all the necessary libraries
import pymzml
import csv
import os
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import re
import plotly.express as px
from collections import defaultdict

import plotly.io as pio
import json
import plotly.graph_objs as go
import matplotlib.colors as mcolors

import json
import ipywidgets as widgets
from IPython.display import display

import warnings
from IPython.display import display, Image, clear_output
import time

##Custom SCripts

from NO_AVERAGE_SCRIPTS import full_parse
from NO_AVERAGE_SCRIPTS import filter_dataframe
from NO_AVERAGE_SCRIPTS import hex_to_rgba_hex
from NO_AVERAGE_SCRIPTS import json_to_string
from NO_AVERAGE_SCRIPTS import prep_edge_R

from NO_AVERAGE_SCRIPTS import make_pie_chart_no_replicates

from NO_AVERAGE_SCRIPTS import average_pie_chart_no_repeats

from NO_AVERAGE_SCRIPTS import make_bar_plot_comparisons
import os
from IPython.display import display
from ipywidgets import widgets
# dfdfplotting_functions

Pre_folder = './Projects/'


from tools.GUI import assign_blank, load_blank_name
from tools.GUI import folder_navigator, load_project_folder

from tools.GUI import filter_samples, load_data_labels
from tools.GUI import display_pair_widgets, load_data, remove_empty_entries, get_unique_json_objects


from tools.parsing import add_suffix

In [2]:
##Add a way to include a custom color scheme




file_name_to_save = 'CLAW_DTRA_3bac'

extra_name = ""

tolerance = 0.1
remove_std = True
save_data= True


load_previously_parsed = True
custom_data=True


In [4]:

folder_navigator()

Project_Folder = load_project_folder()
folder_name_to_save = Project_Folder+ 'Processed Results/'
data_base_name_location = 'lipid_database/Lipid_Database.xlsx'
mzml_folder = Project_Folder +"mzml/"
Pre_edge_r_path = Project_Folder+"Pre_EdgeR/"
plots_2_save_path = Project_Folder+"Plots/"
label_file = Project_Folder+"Labels/labels.csv"
labels_df = pd.read_csv(label_file)

##determines blank
unique_samples = labels_df['Sample Name'].unique()
assign_blank(unique_samples)
blank_name = load_blank_name()

# Assuming labels_df is your DataFrame
filter_samples(labels_df)
labels_df2 = load_data_labels()
labels_df2 = labels_df2.drop(["Sample Name","Position"],axis=1)
labels_list = list(labels_df)
labels_list = labels_list +["Class","Lipid"]





Button(description='Navigate', style=ButtonStyle())

Button(description='Select this folder', style=ButtonStyle())

Button(description='Select Current Folder', style=ButtonStyle())

Select(options=('/home/iyer95/CLAW/CLAW/lipid_platform/Projects',), rows=10, value='/home/iyer95/CLAW/CLAW/lip…

Output()

...

Button(description='Assign Blank', style=ButtonStyle())

Output()

SelectMultiple(description='Samples', index=(0,), options=('A-T4-0-BDL-r001', 'C-T4-8GEN-BDL-r003', 'B-T4-0-BD…

Button(description='Filter Samples', style=ButtonStyle())

Output()

In [5]:
main_json = {col: labels_df2[col].unique().tolist() for col in labels_df2}

display_pair_widgets(main_json)

In [6]:
# Ignore warnings
warnings.filterwarnings('ignore')

# # Display GIF
# gif = Image(filename='Figures/cat_gif.gif') # replace 'your_gif.gif' with the path to your GIF
# display(gif)

print("Your data is PURRing...")

if load_previously_parsed == True:
    df_matched = pd.read_csv(Project_Folder+"Processed Results/"+file_name_to_save+".csv")
    

    
#     tag_values = ['TAG14', 'TAG16', 'TAG18', 'TAG20', 'TAG22']
#     dag_values = ['DAG16', 'DAG18']

#     df_matched.loc[df_matched['method_type'].isin(tag_values), 'Class'] = 'TAG'
#     df_matched.loc[df_matched['method_type'].isin(dag_values), 'Class'] = 'DAG'
    
else:
    df_matched = full_parse(data_base_name_location,mzml_folder, folder_name_to_save,labels_df, blank_name, 
                            file_name_to_save,tolerance, custom_data = custom_data, remove_std = remove_std,save_data=save_data)

print("Complete")

# df_matched = add_labels(labels_df,df_matched)
# merged_df = subtract_blank(labels_df,df_matched,remove_list,blank_name)
df_matched
if custom_data ==True: ###Need to fix for custom classes
    df_matched['Class'] = df_matched['Class'].replace({'AC': 'CAR', 'FFA': 'FA', 'CE | CE': 'CE', 'PE | PE': 'PE', 'PG | PG': 'PG'
                                                      , 'PG | PG | PG': 'PG', 'PI | PI': 'PI', 'PS | PS': 'PS','CER': 'Cer'})
    




json_list_pairs = load_data()
json_list_pairs = remove_empty_entries(json_list_pairs)
json_list_singles = get_unique_json_objects(json_list_pairs)
print(json_list_pairs)
print(json_list_singles)
# make_pie_chart_no_replicates(df_matched,plots_2_save_path,json_list_singles,labels_list,blank_name,extra_name)
# average_pie_chart_no_repeats(df_matched,plots_2_save_path,json_list_singles,labels_list,blank_name,extra_name)
# make_bar_plot_comparisons(df_matched, plots_2_save_path, json_list_pairs,labels_list,blank_name,extra_name)
labels_list = labels_list + ['method_type',"Transition"]
df_matched = add_suffix(df_matched)
print(df_matched)
combined_df = prep_edge_R(df_matched,json_list_pairs,Pre_edge_r_path,blank_name,labels_list,extra_name)
# !bash myjob.sh


Your data is PURRing...
Complete
[[{'Time': ['T2'], 'Dose': ['8'], 'Bacteria': ['Bsub']}, {'Time': ['T0'], 'Dose': ['0'], 'Bacteria': ['Bsub']}], [{'Time': ['T2'], 'Dose': ['8'], 'Bacteria': ['Bsub']}, {'Time': ['T2'], 'Dose': ['0'], 'Bacteria': ['Bsub']}], [{'Time': ['T4'], 'Dose': ['8'], 'Bacteria': ['Bsub']}, {'Time': ['T4'], 'Dose': ['0'], 'Bacteria': ['Bsub']}], [{'Time': ['T6'], 'Dose': ['8'], 'Bacteria': ['Bsub']}, {'Time': ['T6'], 'Dose': ['8'], 'Bacteria': ['Bsub']}], [{}, {}]]
[{'Time': ['T4'], 'Dose': ['0'], 'Bacteria': ['Bsub']}, {'Time': ['T6'], 'Dose': ['8'], 'Bacteria': ['Bsub']}, {'Time': ['T4'], 'Dose': ['8'], 'Bacteria': ['Bsub']}, {}, {'Time': ['T2'], 'Dose': ['0'], 'Bacteria': ['Bsub']}, {'Time': ['T0'], 'Dose': ['0'], 'Bacteria': ['Bsub']}, {'Time': ['T2'], 'Dose': ['8'], 'Bacteria': ['Bsub']}]
      Class              Lipid  Intensity  Precursor_Ion  Product_Ion  \
0        DG     DG(32:0)_C16:0    14237.0          586.5        313.2   
1        DG     DG(32:1)_C1

In [23]:
# # Drop the columns 'Position' and 'Sample Name'
# df_matched = df_matched.drop(columns=['Position', 'Sample Name'])
#drop method_type column
# df_matched = df_matched.drop(columns=['method_type'])
# df_matched = df_matched.rename(columns={'Parent_Ion': 'Precursor_Ion'})

# # Replace "TAG" with "TG"
# df_matched['Lipid'] = df_matched['Lipid'].str.replace('TAG', 'TG')

# # Change values in the 'Class' column
# df_matched['Class'] = df_matched['Class'].replace({'TAG': 'TG', 'DAG': 'DG'})

# # Remove rows where 'Sample_ID' contains "Blank"
# df_matched = df_matched[~df_matched['Sample_ID'].str.contains('Blank')]
df_matched["Intensity"] = df_matched["Intensity"].round(0)
df_matched["Blank Subtraction"] = df_matched["Blank Subtraction"].round(0)


df_matched.head(None)

,Lipid,Precursor_Ion,Product_Ion,Intensity,Transition,Class,Sample_ID,Genotype,Brain Region,Sex,Blank Subtraction
4491,"(5Z,8Z)-tetradecadienoylcarnitine",368.3,85.1,97017264.0,368.3 -> 85.1,CAR,AC_DOD100-M1-WT-cerebellum10x_01202023,WT,cerebellum,Male,95231340.0
4492,cis-5-Tetradecenoylcarnitine,370.3,85.1,2996571.0,370.3 -> 85.1,CAR,AC_DOD100-M1-WT-cerebellum10x_01202023,WT,cerebellum,Male,1506027.0
4493,TetradecanoylcarnitineO-tetradecanoylcarnitine,372.3,85.1,1358860.0,372.3 -> 85.1,CAR,AC_DOD100-M1-WT-cerebellum10x_01202023,WT,cerebellum,Male,0.0
4494,O-(11-carboxyundecanoyl)carnitine,374.2,85.1,1636443.0,374.2 -> 85.1,CAR,AC_DOD100-M1-WT-cerebellum10x_01202023,WT,cerebellum,Male,204025.0
4495,"(5Z,8Z)-3-hydroxytetradecadienoylcarnitine",384.3,85.1,2215148.0,384.3 -> 85.1,CAR,AC_DOD100-M1-WT-cerebellum10x_01202023,WT,cerebellum,Male,137568.0
...,...,...,...,...,...,...,...,...,...,...,...
50893,TG(60:1)_FA 18:2,990.9,693.9,18610.0,990.9 -> 693.9,TG,TAG2_M5FAD173-WT-Hippo10x_01192023,WT,hippo,Male,0.0
50894,TG(60:0)_FA 20:0,992.9,663.9,113377.0,992.9 -> 663.9,TG,TAG2_M5FAD173-WT-Hippo10x_01192023,WT,hippo,Male,6108.0
50895,TG(60:0)_FA 20:4,992.9,671.9,15025.0,992.9 -> 671.9,TG,TAG2_M5FAD173-WT-Hippo10x_01192023,WT,hippo,Male,0.0
50896,TG(60:0)_FA 18:1,992.9,693.9,17602.0,992.9 -> 693.9,TG,TAG2_M5FAD173-WT-Hippo10x_01192023,WT,hippo,Male,0.0


In [24]:
df_matched.to_csv('CLAW_Brain_MRM.csv', index=False)
df_matched.to_excel('CLAW_Brain_MRM.xlsx', index=False)